# Getting The Data
Brian Bahmanyar

___

** NOTE:** This notebook will not run without implementing your own get_token function to return a Quandl API token
* **just use the data from ./data**

[Quandl](https://www.quandl.com) provides free daily financial data which will be used in the analyses to come. They also provide a free, but somewhat lackluster Python API.

In [ ]:
import numpy as np
import pandas as pd
import Quandl
from my_token import get_token

> get_token is a function that returns my Quandl API token, replace with your information as necessary. Or just use the data below, provided in /data.

Below is a function written to serve as a wrapper around Quandl's Python API and provide some needed functionality.

In [ ]:
def get_adj_close(token, tickers, start, end="", ratios=[], log_transforms=[]):
    """
    Args:
        tickers (list): collection of ticker symbols for which to collect adj. close 
                daily prices for
        start (string, format: 2013-01-01): start date for which to collect prices after
        end (string, format: 2013-01-01): optional end date, today if not specified
        ratios (list): collection of tuples of tickers from 'tickers' list to calculate 
                price ratios for (the stock with larger mean is numerator)
        log_transforms (list): collection of tickers from 'tickers' to include additional 
                natural log transformed copies
    
    Returns (dataframe): all adj. close prices, ratios, and log transforms specified
    """ 
    result = {}
    
    for ticker in tickers:
        try:
            result[ticker] = Quandl.get('WIKI/'+ticker, trim_start=start, trim_end=end, authtoken=token)['Adj. Close']
        except DatasetNotFound:
            print('ERROR:')
            print(ticker, 'is not a vaild ticker')

    for ratio in ratios:
        try:
            ticker1, ticker2 = ratio
            if result[ticker1].mean() > result[ticker2].mean():
                result[ticker1+'/'+ticker2] = result[ticker1]/result[ticker2]
            else:
                result[ticker2+'/'+ticker1] = result[ticker2]/result[ticker1]
        except KeyError:
            print('ERROR:')
            print(ticker1, 'or', ticker2, 'are not in the list of specified tickers')
    
    for log_transform in log_transforms:
        try:
            result['ln('+log_transform+')'] = np.log(result[log_transform])
        except KeyError:
            print('ERROR:')
            print(log_transform, 'is not in the list of specified tickers')
    
    return pd.DataFrame(result).dropna() # drop na here because of differences in lenght of history for stocks

> A copy of this function is placed into api_wrapper.py for use in other notebooks.

***

### Now to Get the Data We Need

In [ ]:
tech_bundle = get_adj_close(get_token(),
                            ['FB', 'AMZN', 'AAPL'], 
                            start='2013-01-01', 
                            ratios=[('FB','AMZN'), ('FB','AAPL'), ('AMZN','AAPL')],
                            log_transforms=['FB', 'AMZN', 'AAPL'])

In [ ]:
pairs_bundle = get_adj_close(get_token(),
                            ['VZ', 'T', 'KO', 'PEP', 'XOM', 'CVX'], 
                            start='1990-01-01', 
                            ratios=[('VZ','T'), ('KO','PEP'), ('XOM','CVX')])

In [ ]:
tech_bundle.head()

In [ ]:
pairs_bundle.head()

In [ ]:
# tech_bundle.to_csv('data/tech_bundle.csv')
# pairs_bundle.to_csv('data/pairs_bundle.csv')

___

In [1]:
from IPython.display import HTML
HTML("""<style>@import "http://fonts.googleapis.com/css?family=Lato|Source+Code+Pro|Montserrat:400,700";@font-face{font-family:"Computer Modern";src:url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf')}.rendered_html h1,h2,h3,h4,h5,h6,p{font-family:'Computer Modern'}p,ul{font-family:'Computer Modern'}div#notebook-container{-webkit-box-shadow:none;box-shadow:none}h1{font-size:70pt}h2{font-size:50pt}h3{font-size:40pt}h4{font-size:35pt}h5{font-size:30pt}h6{font-size:25pt}.rendered_html p{font-size:11pt;line-height:14pt}.CodeMirror pre{font-family:'Source Code Pro', monospace;font-size:09pt}div.input_area{border:none;background:#f5f5f5}ul{font-size:10.5pt;font-family:'Computer Modern'}ol{font-size:11pt;font-family:'Computer Modern'}.output_png img{display:block;margin-left:auto;margin-right:auto}</style>""")